<a href="https://colab.research.google.com/github/wanyoike274/Mxnet-Projects/blob/main/Diabete_Classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import mxnet as mx
import pandas as pd

# Load the CSV file
url = "http://aka.ms/diabetes-data"
data = pd.read_csv(url)

# Separate features and labels
features = data.iloc[:, :-1].values
labels = data.iloc[:, -1].values

# Convert labels to numerical values
label_map = {label: i for i, label in enumerate(set(labels))}
labels = [label_map[label] for label in labels]

# Define the network architecture
net = mx.gluon.nn.Sequential()
with net.name_scope():
    net.add(mx.gluon.nn.Dense(64, activation='relu'))
    net.add(mx.gluon.nn.Dense(32, activation='relu'))
    net.add(mx.gluon.nn.Dense(2))

# Initialize the network parameters
net.initialize(mx.init.Xavier(magnitude=2.24))

# Set the context to GPU if available, otherwise use CPU
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()

# Create data iterators for training and validation
batch_size = 32
train_data = mx.gluon.data.DataLoader(mx.gluon.data.ArrayDataset(features, labels),
                                      batch_size=batch_size, shuffle=True)
# Split the dataset into 80% training and 20% validation
train_data, val_data = train_data.random_split(0.8)

# Define the loss function
loss_fn = mx.gluon.loss.SoftmaxCrossEntropyLoss()

# Define the optimizer
optimizer = mx.gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': 0.001})

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = 0
    for data, label in train_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with mx.autograd.record():
            output = net(data)
            loss = loss_fn(output, label)
        loss.backward()
        optimizer.step(data.shape[0])
        train_loss += loss.mean().asscalar()
    
    # Calculate validation accuracy
    val_accuracy = mx.metric.Accuracy()
    for data, label in val_data:
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = net(data)
        predictions = mx.nd.argmax(output, axis=1)
        val_accuracy.update(label, predictions)
    
    print(f'Epoch {epoch+1}: Train Loss {train_loss/len(train_data)}, Val Accuracy {val_accuracy.get()[1]}')

# Save the trained model
net.save_parameters('model.params')


AttributeError: ignored

In [2]:
pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
